### gensium

#### 1、困惑度 perp

这个问题在《LDA漫游指南》一书中做了很好的解答，详见第4章第4.2节

topic number K：

许多读者问，如何设置主题个数，其实现在没有特别好的办（HDP等较为复杂的模型可以自动确定这个参数，但是模型复杂，计算复杂），

$\color{red}{目前只有交叉验证（cross validation），通过设置不同的K值训练后验证比较求得最佳值}$

我的建议是一开始不要设置太大而逐步增大实验，
Blei在论文《Latent Dirichlet Allocation》提出过一个方法，采用设置不同的topic数量，画出topic_number-perplexity曲线；

Thomas L. Grifﬁths等人在《Finding scientific topics》也提出过一个验证方法，画出topic_number-logP(w|T)曲线，

然后找到曲线中的纵轴最高点便是topic数量的最佳值。有兴趣的读者可以去读读这两篇论文原文的相应部分。

这个参数同时也跟文章数量有关，可以通过一个思想实验来验证：

设想两个极端情况：如果仅有一篇文章做训练，则设置几百个topic不合适，

如果将好几亿篇文章拿来做topic model，则仅仅设置很少topic也是不合适的。

（1）Perplexity是什么？

通常用于评价聚类算法好坏的方法有两种，其一是使用带分类标签的测试数据集，然后使用一些算法，比如Normalized Mutual Information,Variation of Information distance,来判断聚类结果与真实结果的差距，其二是使用无分类标签的测试数据集，用训练出来的模型来跑测试数据集，然后计算在测试数据集上，所有token似然值几何平均数的倒数，也就是perplexity指标，这个指标可以直观理解为用于生成测试数据集的词表大小的期望值，而这个词表中所有词汇符合平均分布[1]。其公式如下：

<img src= "img/困惑度.jpg">

### 我们来看一篇英文文档怎么说perplexity

<img src="img/困惑度1.png">

大概意思是说，困惑度是用于评测语言模型中，聚类算法的一个通用方法，它用于评测一个unseen的测试集，基于训练集训练出来的模型结果M。实际上就是求在测试集W上的一个似然函数。，P(W|M)中，W可以是一篇文档、或者每一个词。对于LDA模型而言，通常W是一个word
<img src="img/困惑度2.png">

复杂度越高表示训练出来的模型参数（主要是P(z|d) 和 p(w|d)），对于测试集而言，会有表达出错会更多些。

In [9]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import numpy as np

def perplexity(ldamodel, testset, dictionary, size_dictionary, num_topics):
    """calculate the perplexity of a lda-model"""
    # dictionary : {7822:'deferment', 1841:'circuitry',19202:'fabianism'...]
    # print('the info of this ldamodel: ')
    # print('num of testset: %s; size_dictionary: %s; num of topics: %s' % (len(testset), size_dictionary, num_topics))
    prob_doc_sum = 0.0
    topic_word_list = []
    for topic_id in range(num_topics):
        topic_word = ldamodel.show_topic(topic_id, size_dictionary)
        dic = {}
        for word, probability in topic_word:
            dic[word] = probability
        topic_word_list.append(dic)
    doc_topics_ist = []
    for doc in testset:
        doc_topics_ist.append(ldamodel.get_document_topics(doc, minimum_probability=0))
    testset_word_num = 0
    for i in range(len(testset)):
        prob_doc = 0.0  # the probablity of the doc
        doc = testset[i]
        doc_word_num = 0  # the num of words in the doc
        for word_id, num in doc:
            prob_word = 0.0  # the probablity of the word
            doc_word_num += num
            word = dictionary[word_id]
            for topic_id in range(num_topics):
                # cal p(w) : p(w) = sum_z{(p(z)*p(w|z))}
                try:
                    prob_topic = doc_topics_ist[i][topic_id][1]  # p(z_i) = p(z = i|d)
                except Exception:
                    prob_topic = 0.00001
                # print("prob_topic", prob_topic)
                prob_topic_word = topic_word_list[topic_id][word]
                prob_word += prob_topic*prob_topic_word
            prob_doc += np.log(prob_word) # p(d) = sum(log(p(w)))
        prob_doc_sum += prob_doc
        testset_word_num += doc_word_num
    prep = np.exp2(-prob_doc_sum/testset_word_num) # perplexity = exp(-sum(p(d)/sum(Nd))
    # print("the perplexity of this ldamodel is : %s" % prep)
    return prep

docs = [['human', 'interface', 'computer'],
         ['survey', 'user', 'computer', 'system', 'response', 'time'],
         ['eps', 'user', 'interface', 'system'],
         ['system', 'human', 'system', 'eps'],
         ['user', 'response', 'time'],
         ['trees'],
         ['graph', 'trees'],
         ['graph', 'minors', 'trees'],
         ['graph', 'minors', 'survey']]
dct = Dictionary(docs)
corpus = [dct.doc2bow(_) for _ in docs]
c_train, c_test = corpus[:7], corpus[7:]

ldamodel = LdaModel(corpus=c_train, num_topics=2, id2word=dct)
log_perp =ldamodel.log_perplexity(c_test)
print(log_perp)
print(np.exp2(log_perp))
# print("my perp: ", perplexity(ldamodel, c_test, dct, 9, 2))

-4.863109226028125
0.03436040315132479


#### 2、主题一致性

In [11]:
from gensim.models.coherencemodel import CoherenceModel

texts = [['human', 'interface', 'computer'],
         ['survey', 'user', 'computer', 'system', 'response', 'time'],
         ['eps', 'user', 'interface', 'system'],
         ['system', 'human', 'system', 'eps'],
         ['user', 'response', 'time'],
         ['trees'],
         ['graph', 'trees'],
         ['graph', 'minors', 'trees'],
         ['graph', 'minors', 'survey']]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# 1.1、训练模型
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=100, num_topics=2)
# 1.2、通过预训练好的模型来求coh
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')
print(goodcm.get_coherence())

# 2.1 不提供模型，直接提供topics
topics = [['human', 'computer', 'system', 'interface'],
          ['graph', 'minors', 'trees', 'eps']]

# note that a dictionary has to be provided.
cm = CoherenceModel(topics=topics, corpus=corpus, dictionary=dictionary, coherence='u_mass')
cm.get_coherence()

-14.689200643963385


-7.105015580153772

### 问题
1、perplexity和coherence到底哪个靠谱？？
    
    Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. In my experience, topic coherence score, in particular, has been more helpful. 
    业界普遍认为coherence效果可能更好

### 彻底搞透LDA

#### gensim版

In [2]:
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import re

In [3]:
# 1、加载停用词

stopwords = open("stop_words.txt", "r", encoding="gbk").read().split("\n")

In [4]:
len(stopwords),stopwords[:10]

(1339, ['俺', '如', '呜', '啊', '把', '会', '焉', '地', '因', '冲'])

In [5]:
# 1、文本预处理 tokenize

def tokenize(text):
    text = text.lower()
    words = re.sub("[A-Za-z0-9.?/。,，;；、：:""‘’“”'!！$%()》《()（）—_√□【】+<>-]\n", " ", text).split()
    words = [w for w in words if w not in stopwords and len(w) > 1]
    return words

In [6]:
%%time
# 3、获取语料 

import pymongo
# mongodb
host='dds-bp130b89ecd3ee24-pub.mongodb.rds.aliyuncs.com'
port=3717
db='Fin-Tech'
user='xuyong'
passwd='FinTech_xy'
collect='doc_info_seg_text'

documents=[]
documents_name=[]

# 2010-2017，共8年的财报
client = pymongo.MongoClient(host, port)
db = client[db]
db.authenticate(user, passwd, mechanism='SCRAM-SHA-1')
col = db[collect]
for w in col.find({
    'report_type': 1, 
    'quart_v': {'$in': [4]},
#     'security_code': {'$in': code},
    'report_year': {'$gte': 2016, '$lte': 2016}},
    {'_id': 0, 'seg_text_clean': 1, 'file_name': 1}):
        documents.append(w['seg_text_clean'])
        documents_name.append(w['file_name'])
print('get documents done.')

get documents done.
Wall time: 2min 29s


In [19]:
len(documents),len(documents[0]),len(documents[0].split(" ")),len(tokenize(documents[0]))

(3106, 70843, 21182, 12895)

In [9]:
%%time
# 4、分词、去停用词
processed_docs = [tokenize(doc) for doc in documents]
print('tokenlize done.')

tokenlize done.
Wall time: 20min 17s


In [48]:
%%time
import copy
# 5、生成词表
word_count_dict = Dictionary(processed_docs)


Wall time: 41.5 s


In [50]:
len(word_count_dict)

180378

In [38]:
%%time
cnt = 0
risk = []
for key in word_count_dict.token2id.keys():
    if key.find("风险") is 0:
        print(key)
        risk.append(key)
#     cnt += 1
#     if cnt % 1000 ==0:
#         print(cnt)

风险
风险属性
风险意识
风险控制
风险敞口
风险管理
风险评估
风险因素
风险防范
风险点
风险投资
风险监控
风险教育
风险价值
风险偏好
风险准备金
风险处置
风险容忍度
风险承受能力
风险控制机制
风险提示
风险收益
风险收益分析
风险暴露
风险暴露程度
风险监管
风险管理制度
风险管理委员会
风险管理部
风险类别
风险警示
风险加权资产
风险可控
风险承担
风险指标
风险状况
风险程度
风险经营
风险调整
风险资产
风险较高
风险防控
风险预警
风险导向原则
风险领域
风险较大
风险隐患
风险报酬
风险套利
风险管理系统
风险转移
风险抵押金
风险识别
风险理念
风险计量
风险评级
风险量化
风险防范体系
风险披露
风险等级
风险加大
风险控制委员会
风险性
风险提示函
风险联动
风险问责
风险问题
风险管控体系
风险评价
风险清单
风险保障
风险规避
风险资本
风险最小
风险头寸
风险辨识
风险分析
风险巨大
风险提示公告
风险系数
风险分担
风险内控
风险极低
风险容限
风险成本
风险分散
风险金
风险警示板
风险实质
风险值
风险参数
风险控制系统
风险业务管理
风险事故
风险调整资本回报率
风险经理
风险溢价
风险放大
风险自负
风险最大
风险投资基金
风险咨询
风险调整回报率
风险要素
风险投资公司
风险信息化
风险厌恶
Wall time: 83 ms


In [39]:
len(risk)

107

In [54]:
%%time
# 5.1 过滤一些高频和低频次
# no_below : int, optional
#             Keep tokens which are contained in at least `no_below` documents.  至少要有no_below篇doc中，出现这个token
# 一般选
word_count_dict.filter_extremes(no_below=2, no_above=0.9, keep_n=200000)

Wall time: 566 ms


In [55]:
len(word_count_dict)

90174

In [56]:
%%time
risk_2 = []
for key in word_count_dict.token2id.keys():
    if key.find("风险") is 0:
        risk_2.append(key)

Wall time: 46.9 ms


In [58]:
set(risk) - set(risk_2)

{'风险',
 '风险业务管理',
 '风险信息化',
 '风险厌恶',
 '风险参数',
 '风险咨询',
 '风险容限',
 '风险放大',
 '风险自负',
 '风险要素',
 '风险警示板',
 '风险评估',
 '风险调整回报率'}